In [1]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, TfidfTransformer

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

In [2]:
## ML Library
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import RepeatedStratifiedKFold,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
import pickle

## Visualization library
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud

## Ignoring Warning during trainings 
import warnings
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
train = train.drop(['id', 'keyword','location'], axis = 1) 

In [6]:
# lowering the text
train.text=train.text.apply(lambda x:x.lower() )
test.text=test.text.apply(lambda x:x.lower())
#removing square brackets
train.text=train.text.apply(lambda x:re.sub('\[.*?\]', '', x) )
test.text=test.text.apply(lambda x:re.sub('\[.*?\]', '', x) )
train.text=train.text.apply(lambda x:re.sub('<.*?>+', '', x) )
test.text=test.text.apply(lambda x:re.sub('<.*?>+', '', x) )
#removing hyperlink
train.text=train.text.apply(lambda x:re.sub('https?://\S+|www\.\S+', '', x) )
test.text=test.text.apply(lambda x:re.sub('https?://\S+|www\.\S+', '', x) )
#removing puncuation
train.text=train.text.apply(lambda x:re.sub('[%s]' % re.escape(string.punctuation), '', x) )
test.text=test.text.apply(lambda x:re.sub('[%s]' % re.escape(string.punctuation), '', x) )
train.text=train.text.apply(lambda x:re.sub('\n' , '', x) )
test.text=test.text.apply(lambda x:re.sub('\n', '', x) )
#remove words containing numbers
train.text=train.text.apply(lambda x:re.sub('\w*\d\w*' , '', x) )
test.text=test.text.apply(lambda x:re.sub('\w*\d\w*', '', x) )

In [7]:
#Tokenizer
token=nltk.tokenize.RegexpTokenizer(r'\w+')
#applying token
train.text=train.text.apply(lambda x:token.tokenize(x))
test.text=test.text.apply(lambda x:token.tokenize(x))

In [8]:
nltk.download('stopwords')
#removing stop words
train.text=train.text.apply(lambda x:[w for w in x if w not in stopwords.words('english')])
test.text=test.text.apply(lambda x:[w for w in x if w not in stopwords.words('english')])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pragya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
#stemmering the text and joining
stemmer = nltk.stem.PorterStemmer()
train.text=train.text.apply(lambda x:" ".join(stemmer.stem(token) for token in x))
test.text=test.text.apply(lambda x:" ".join(stemmer.stem(token) for token in x))

In [10]:
count_vectorizer = CountVectorizer()
train_vectors_count = count_vectorizer.fit_transform(train['text'])
test_vectors_count = count_vectorizer.transform(test["text"])

In [ ]:
test = test.drop(['text', 'keyword','location'], axis = 1) 

In [13]:
model1 = MultinomialNB()
model1.fit(train_vectors_count, train["target"])
pred1= model1.predict(test_vectors_count)

In [14]:
pred1.sum()

1263

In [ ]:
# test['target'] = pred1
# ## NAIVE BAYES
# test.to_csv(r'submission1.csv', index = False)

In [ ]:
model2 = LogisticRegression()
model2.fit(train_vectors_count, train["target"])
pred2=model2.predict(test_vectors_count)

In [ ]:
pred2.sum()

In [ ]:
# test = test.drop(['target'], axis = 1) 
# test['target'] = pred2
# ## LOGISTIC
# test.to_csv(r'submission2.csv', index = False)

In [17]:
from sklearn.tree import DecisionTreeClassifier
model4 = DecisionTreeClassifier()
model4 = model4.fit(train_vectors_count, train["target"])
pred4 = model4.predict(test_vectors_count)

In [18]:
pred4.sum()

1252

In [ ]:
# test = test.drop(['target'], axis = 1) 
# test['target'] = pred4
# ## DECISION TREE
# test.to_csv(r'submission4.csv', index = False)

In [19]:
#import required packages
from sklearn import neighbors
from sklearn.metrics import mean_squared_error 
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
# KNN
model5 = neighbors.KNeighborsRegressor(n_neighbors = 2)
model5.fit(train_vectors_count, train["target"])  #fit the model
pred5=model5.predict(test_vectors_count) #make prediction on test set
predd5 = [1 if i>=0.5 else 0 for i in pred5]
pred5 = np.array(predd5)
pred5.sum()

1228

In [ ]:
# test = test.drop(['target'], axis = 1) 
# test['target'] = pred5
# ## KNN
# test.to_csv(r'submission5.csv', index = False)

In [69]:
from scipy import stats
final_pred = np.array([])
for i in range(0,3263):
    final_pred = np.append(final_pred, stats.mode([pred1[i], pred2[i], pred4[i]])[0])

In [66]:
test = test.drop(['target'], axis = 1)
final_pred = [int(i) for i in final_pred]
test['target'] = final_pred
test.to_csv(r'submission7.csv', index = False) ## ENSEMBLING (MAJORITY VOTING)

In [70]:
final_pred.sum()

1175.0